# Carregamento da base de dados

In [2]:
#Importando Bibiliotecas
import pandas as pd
print('Importando Bibliotecas...')
import time
import psycopg2
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine
import urllib
print('Bibliotecas importadas')

Importando Bibliotecas...
Bibliotecas importadas


In [36]:
#Baixando dados da anatel de banda larga
import wget
urlbandalarga ='https://www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_banda_larga_fixa.zip'
wget.download(urlbandalarga)

100% [......................................................................] 660081702 / 660081702

'acessos_banda_larga_fixa.zip'

#Download dos dados do site do INEP.
import requests
import urllib3
from zipfile import ZipFile 
urllib3.disable_warnings()

headers={
  'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,/;q=0.8',
  'Accept-Language':'zh-CN,zh;q=0.9,en;q=0.8,ja;q=0.7',
  'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'
};


nome_arquivo = "meu_municipio.zip"
url =  "https://www.anatel.gov.br/dadosabertos/paineis_de_dados/meu_municipio/meu_municipio.zip"
   
      
r = requests.get(url, headers=headers, verify=False)
with open(nome_arquivo, "wb") as f:
    for chunk in r.iter_content(chunk_size=512): 
        if chunk:
            f.write(chunk) 

# Pré-Analise dos dados

In [63]:
#Descompactação dos arquivos baixados em uma pasta chamada outputanatelacesos/bandalargacolunas/bandalarga.
import os
import re
from zipfile import ZipFile

if os.path.isdir("outputanatelacessos/bandalarga/colunas"):
    print("O diretório outputanatelacessos já existe!")
else:
    print("O diretório outputanatelacessos não existe!, criando diretório")
    
    banda_larga = "acessos_banda_larga_fixa.zip"

    with ZipFile(banda_larga, 'r') as zip: 
        zip.extractall('outputanatelacessos/bandalargacolunas/bandalarga') 
        print('Feito!') 


O diretório outputanatelacessos não existe!, criando diretório
Feito!


In [59]:
#Movendo os arquivos para as suas respectivas pastas
import os

lista_arquivos = os.listdir("outputanatelacessos/bandalargacolunas/bandalarga/")

for arquivo in lista_arquivos:
            if 'Colunas' in arquivo:
                os.rename('outputanatelacessos/bandalargacolunas/bandalarga/' + arquivo, f'outputanatelacessos/bandalargacolunas/'+arquivo)


       

In [60]:
#Concatenando os dados de acesso banda larga fixa do estado do Pará
import glob
import pandas as pd
arquivos = glob.glob('outputanatelacessos/bandalargacolunas/bandalarga/Acessos_Banda_Larga_Fixa_2*')
array_df = []

for x in arquivos:
    temp_df = pd.read_csv(x ,encoding="UTF-8", sep=';', low_memory=False)
    dataframe = pd.DataFrame(temp_df)
    rslt_df = dataframe[temp_df['UF'] == 'PA' ]
    array_df.append(rslt_df)

df = pd.concat(array_df, axis=0, )

print(df)


          Ano  Mês   Grupo Econômico Empresa            CNPJ   
243      2010   12  TELECOM AMERICAS   CLARO  33530486000129  \
323      2010   12  TELECOM AMERICAS   CLARO  33530486000129   
340      2010   12  TELECOM AMERICAS   CLARO  33530486000129   
379      2010   12  TELECOM AMERICAS   CLARO  33530486000129   
414      2010   12  TELECOM AMERICAS   CLARO  33530486000129   
...       ...  ...               ...     ...             ...   
1144359  2023    1                OI      OI  76535764000143   
1144361  2023    1                OI      OI  76535764000143   
1144461  2023    1                OI      OI  76535764000143   
1144565  2023    1                OI      OI  76535764000143   
1144571  2023    1                OI      OI  76535764000143   

        Porte da Prestadora  UF     Município  Código IBGE Município   
243            Grande Porte  PA     Castanhal                1502400  \
323            Grande Porte  PA         Soure                1507904   
340            

# Ajuste na tipagem dos dados

In [61]:
#deixando as colunas em caixa baixa e substituindo espaços 
from unidecode import unidecode
teste = list(df.columns)
colunas =[]
for x in teste:
    semmaiuscula = unidecode(x.lower())
    repl = semmaiuscula.replace(" ", "_")
    print(repl)
    colunas.append(repl)
    
print(colunas)

df.columns = colunas
print(df)

ano
mes
grupo_economico
empresa
cnpj
porte_da_prestadora
uf
municipio
codigo_ibge_municipio
faixa_de_velocidade
tecnologia
meio_de_acesso
acessos
velocidade
tipo_de_pessoa
tipo_de_produto
['ano', 'mes', 'grupo_economico', 'empresa', 'cnpj', 'porte_da_prestadora', 'uf', 'municipio', 'codigo_ibge_municipio', 'faixa_de_velocidade', 'tecnologia', 'meio_de_acesso', 'acessos', 'velocidade', 'tipo_de_pessoa', 'tipo_de_produto']
          ano  mes   grupo_economico empresa            cnpj   
243      2010   12  TELECOM AMERICAS   CLARO  33530486000129  \
323      2010   12  TELECOM AMERICAS   CLARO  33530486000129   
340      2010   12  TELECOM AMERICAS   CLARO  33530486000129   
379      2010   12  TELECOM AMERICAS   CLARO  33530486000129   
414      2010   12  TELECOM AMERICAS   CLARO  33530486000129   
...       ...  ...               ...     ...             ...   
1144359  2023    1                OI      OI  76535764000143   
1144361  2023    1                OI      OI  76535764000143   

In [64]:
#Alterando tipo de dados para tipo certo
df.replace({None: 0}, regex=True, inplace=True)

data_types_dict = {'ano': int}
data_types_dict1 = {'mes': int}

df = df.astype(data_types_dict)
df = df.astype(data_types_dict1)


df.dtypes


ano                        int32
mes                        int32
grupo_economico           object
empresa                   object
cnpj                       int64
porte_da_prestadora       object
uf                        object
municipio                 object
codigo_ibge_municipio      int64
faixa_de_velocidade       object
tecnologia                object
meio_de_acesso            object
acessos                  float64
velocidade                object
tipo_de_pessoa            object
tipo_de_produto           object
dtype: object

In [65]:
#Alterando colunas para nomes minusculos
df['empresa'] = df['empresa'].str.lower()
df['tipo_de_produto'] = df['tipo_de_produto'].str.lower()
df['grupo_economico'] = df['grupo_economico'].str.lower()
df.head()

,ano,mes,grupo_economico,empresa,cnpj,porte_da_prestadora,uf,municipio,codigo_ibge_municipio,faixa_de_velocidade,tecnologia,meio_de_acesso,acessos,velocidade,tipo_de_pessoa,tipo_de_produto
243,2010,12,telecom americas,claro,33530486000129,Grande Porte,PA,Castanhal,1502400,512kbps a 2Mbps,xDSL,Cabo Metálico,25.0,0,0,NaN
323,2010,12,telecom americas,claro,33530486000129,Grande Porte,PA,Soure,1507904,0Kbps a 64Kbps,xDSL,Cabo Metálico,3.0,0,0,NaN
340,2010,12,telecom americas,claro,33530486000129,Grande Porte,PA,Soure,1507904,64Kbps a 512Kbps,xDSL,Cabo Metálico,1.0,0,0,NaN
379,2010,12,telecom americas,claro,33530486000129,Grande Porte,PA,Paragominas,1505502,512kbps a 2Mbps,xDSL,Cabo Metálico,35.0,0,0,NaN
414,2010,12,telecom americas,claro,33530486000129,Grande Porte,PA,Tucuruí,1508100,512kbps a 2Mbps,xDSL,Cabo Metálico,2.0,0,0,NaN


In [66]:
df.replace({'win time ltda': 'Flexanet'}, regex=True, inplace=True)

In [67]:
df.replace({'ags telecom ltda - me': 'Link One Telecom'}, regex=True, inplace=True)


In [68]:
df.replace({'coelho tecnologia eireli': 'Coelho Tecnologia'}, regex=True, inplace=True)


In [69]:
df.replace({'garra telecom ltda me': 'Garra Telecom'}, regex=True, inplace=True)


In [70]:
df.replace({'concourse telecomunicacoes brasil ltda.': 'Concourse Telecomunicações brasil'}, regex=True, inplace=True)


In [71]:
df.replace({'jupiter telecomunicacoes e informatica ltda': 'Júpiter'}, regex=True, inplace=True)


In [72]:
df.replace({'oi': 'Oi'}, regex=True, inplace=True)


In [73]:
df.replace({'rede de telecomunicacoes carajas eireli': 'CarajásNet'}, regex=True, inplace=True)


In [74]:

df.replace({'velocitynet telecom eireli - epp': 'VelocyNet'}, regex=True, inplace=True)

In [75]:
df.replace({'wkve-asses. em serv. de inf. e telecomunicacoes ltda': 'WKVE'}, regex=True, inplace=True)

In [76]:
df.head()

,ano,mes,grupo_economico,empresa,cnpj,porte_da_prestadora,uf,municipio,codigo_ibge_municipio,faixa_de_velocidade,tecnologia,meio_de_acesso,acessos,velocidade,tipo_de_pessoa,tipo_de_produto
243,2010,12,telecom americas,claro,33530486000129,Grande Porte,PA,Castanhal,1502400,512kbps a 2Mbps,xDSL,Cabo Metálico,25.0,0,0,NaN
323,2010,12,telecom americas,claro,33530486000129,Grande Porte,PA,Soure,1507904,0Kbps a 64Kbps,xDSL,Cabo Metálico,3.0,0,0,NaN
340,2010,12,telecom americas,claro,33530486000129,Grande Porte,PA,Soure,1507904,64Kbps a 512Kbps,xDSL,Cabo Metálico,1.0,0,0,NaN
379,2010,12,telecom americas,claro,33530486000129,Grande Porte,PA,Paragominas,1505502,512kbps a 2Mbps,xDSL,Cabo Metálico,35.0,0,0,NaN
414,2010,12,telecom americas,claro,33530486000129,Grande Porte,PA,Tucuruí,1508100,512kbps a 2Mbps,xDSL,Cabo Metálico,2.0,0,0,NaN
